In [ ]:
!pip install -q Cython
!pip install -q nemo_toolkit[all]
!pip install -q pyannote.core
!pip install -q pyannote.metrics

import json
import os
from nemo.collections.asr.models import ClusteringDiarizer
from omegaconf import OmegaConf

In [ ]:
def read_num_speakers_from_rttm(rttm_file_path):
    speakers = set()
    with open(rttm_file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            # Ідентифікатор спікера зазвичай знаходиться на 8-й позиції (починаючи з 0)
            speaker_id = parts[7]
            speakers.add(speaker_id)
    return len(speakers)

In [ ]:
import json
import os
from nemo.collections.asr.models import ClusteringDiarizer
from omegaconf import OmegaConf

# Шлях до папок
wav_folder = 'voxconverse_test_wav'
rttm_folder = 'voxconverse_test_rttm'
manifest_folder = 'nemo_manifests'
output_dir_base = 'nemo_output_directory'

# Створення папки для маніфестів, якщо вона ще не існує
os.makedirs(manifest_folder, exist_ok=True)

# Перебір усіх wav файлів у папці
for wav_file in os.listdir(wav_folder):
    if wav_file.endswith('.wav'):
        # Визначення шляхів
        wav_path = os.path.join(wav_folder, wav_file)
        rttm_path = os.path.join(rttm_folder, wav_file.replace('.wav', '.rttm'))
        manifest_path = os.path.join(manifest_folder, wav_file.replace('.wav', '.json'))

        # Читання кількості спікерів з RTTM файла (припустимо, що ви вже маєте функцію для цього)
        num_speakers = read_num_speakers_from_rttm(rttm_path)

        # Створення маніфесту для кожного wav файлу
        meta = {
            'audio_filepath': wav_path,
            'offset': 0,
            'duration': None,
            'label': 'infer',
            'text': '-',
            'num_speakers': num_speakers,
            'rttm_filepath': rttm_path,
            'uem_filepath' : None
        }
        with open(manifest_path, 'w') as fp:
            json.dump(meta, fp)
            fp.write('\n')

        # Шлях до вашого існуючого .yaml файлу
        MODEL_CONFIG = 'diar_infer_general.yaml'

        # Завантаження існуючої конфігурації
        config = OmegaConf.load(MODEL_CONFIG)

        # Оновлення шляху до маніфесту та вихідної директорії
        manifest_path = os.path.join(manifest_folder, wav_file.replace('.wav', '.json'))
        output_dir = os.path.join(output_dir_base, wav_file.replace('.wav', ''))

        config.diarizer.manifest_filepath = manifest_path
        config.diarizer.out_dir = output_dir

        # Виконання діаризації з оновленою конфігурацією
        sd_model = ClusteringDiarizer(cfg=config)
        sd_model.diarize()

[NeMo W 2023-11-22 16:16:08 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


[NeMo I 2023-11-22 16:16:10 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2023-11-22 16:16:10 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2023-11-22 16:16:10 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2023-11-22 16:16:10 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-11-22 16:16:10 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2023-11-22 16:16:10 features:289] PADDING: 16
[NeMo I 2023-11-22 16:16:11 save_restore_connector:249] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2023-11-22 16:16:11 clustering_diarizer:157] Loading pretrained titanet_large model from NGC
[NeMo I 2023-11-22 16:16:11 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.21.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2023-11-22 16:16:11 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.21.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2023-11-22 16:16:11 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-11-22 16:16:11 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-11-22 16:16:11 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

...

[NeMo I 2023-11-22 16:23:52 clustering_diarizer:389] Saved embedding files to nemo_output_directory/crorm/speaker_outputs/embeddings
[NeMo I 2023-11-22 16:23:52 clustering_diarizer:287] Subsegmentation for embedding extraction: scale2, nemo_output_directory/crorm/speaker_outputs/subsegments_scale2.json
[NeMo I 2023-11-22 16:23:52 clustering_diarizer:343] Extracting embeddings for Diarization
[NeMo I 2023-11-22 16:23:53 collections:301] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2023-11-22 16:23:53 collections:302] Dataset loaded with 3633 items, total duration of  0.50 hours.
[NeMo I 2023-11-22 16:23:53 collections:304] # 3633 files loaded accounting to # 1 labels


[3/3] extract embeddings: 100%|██████████| 57/57 [00:04<00:00, 13.74it/s]


[NeMo I 2023-11-22 16:23:58 clustering_diarizer:389] Saved embedding files to nemo_output_directory/crorm/speaker_outputs/embeddings


clustering: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s]

[NeMo I 2023-11-22 16:23:59 clustering_diarizer:464] Outputs are saved in /workspace/nemo_output_directory/crorm directory
[NeMo I 2023-11-22 16:23:59 der:176] Cumulative Results for collar 0.5 sec and ignore_overlap True: 
     FA: 0.0012	 MISS 0.0007	                 Diarization ER: 0.0092	, Confusion ER:0.0073


In [ ]:
import pandas as pd
from pyannote.core import Annotation, Segment
from pyannote.metrics.diarization import DiarizationErrorRate, JaccardErrorRate
from pyannote.metrics.identification import IdentificationErrorRate
from pyannote.metrics.detection import DetectionErrorRate

def load_rttm(file_path):
    annotation = Annotation()
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            start = float(parts[3])
            duration = float(parts[4])
            label = parts[7]
            annotation[Segment(start, start + duration)] = label
    return annotation

rttm_folder = 'voxconverse_test_rttm'  # Шлях до папки з RTTM файлами
file_names = []  # Список для зберігання назв файлів

# Перебір всіх файлів у папці
for file in os.listdir(rttm_folder):
    if file.endswith('.rttm'):
        # Додавання назви файлу без розширення .rttm до списку
        file_names.append(os.path.splitext(file)[0])

# Створення екземпляра DiarizationErrorRate
der_metric = DiarizationErrorRate(collar=0.5)
jer_metric = JaccardErrorRate(collar=0.5)
ier_metric = IdentificationErrorRate(collar=0.5)

# Масив для зберігання значень DER для кожної пари
error_rates_der = []
error_rates_jer = []
error_rates_ier = []

for file_name in file_names:
    reference_path = f'voxconverse_test_rttm/{file_name}.rttm'
    hypothesis_path = f'nemo_output_directory/{file_name}/pred_rttms/{file_name}.rttm'

    reference_annotation = load_rttm(reference_path)
    hypothesis_annotation = load_rttm(hypothesis_path)

    # Calculate metrics
    der = der_metric(reference_annotation, hypothesis_annotation)
    jer = jer_metric(reference_annotation, hypothesis_annotation)
    ier = ier_metric(reference_annotation, hypothesis_annotation)

    error_rates_der.append(der)
    error_rates_jer.append(jer)
    error_rates_ier.append(ier)

# Розрахунок середнього DER
average_der = sum(error_rates_der) / len(error_rates_der)
print(f"Average Diarization Error Rate: {average_der:.2f}")

average_jer = sum(error_rates_jer) / len(error_rates_jer)
print(f"Average Jaccard Error Rate: {average_jer:.2f}")

average_ier = sum(error_rates_ier) / len(error_rates_ier)
print(f"Average Insertion Error Rate: {average_ier:.2f}")

df_result = pd.DataFrame([{
    'Python Library': 'SpeechBrain',
    'Average Diarization Error Rate': average_der,
    'Average Jaccard Error Rate': average_jer,
    'Average Insertion Error Rate': average_ier,
}])

display(df_result)

Average Diarization Error Rate: 0.14
Average Jaccard Error Rate: 0.39
Average Insertion Error Rate: 1.03


,Python Library,Average Diarization Error Rate,Average Jaccard Error Rate,Average Insertion Error Rate
0,SpeechBrain,0.135694,0.3889,1.025886
